# Frida脱壳实战
[android内核集成Frida,Android安全指南 之 Frida脱壳实战](https://blog.csdn.net/weixin_29887351/article/details/117732908)  
[基于Frida的脱壳工具frida-unpack](https://github.com/dstmath/frida-unpack)  




## 一、Frida技术原理
　　Android脱壳的目的是从内存中dump下解密的应用dex文件，为了实现这个目的我们需要知道dex文件在内存中dex地址与dex文件大小。Android系统的libart.so库文件中提供了一个导出的OpenMemory函数用来加载dex文件。  
![OpenMemory函数](./images/Frida脱壳实战-OpenMemory.png)   
　　这个函数的第一个参数指向了内存中的dex文件，如果我们可以Hook 这个函数，就可以得到dex文件加载进内存时的起始地址，再根据dex文件格式计算得到文件头保存的dex文件的长度fileSize。   
　　根据dex文件格式，从起始位置开始第32个字节是该dex文件的大小，知道dex起始位置和整个文件的大小，只是要把这段内存dump出来即可。实用与安卓6、7、8、9。

## 二、获取函数签名
### 2.1、方法一
　　在编写Frida脚本之前我们需要在libart.so文件中找到OpenMemory的函数签名，这个签名根据android版本或者架构的不同会有些许差异，下面介绍如何从运行待脱壳应用的设备中提取libart.so文件：
```shell
$ adb pull /system/lib/libart.so /本地目录
$ adb pull /system/lib64/libart.so /本地目录
```
　　获取libart.so文件后有两个方法可以获得函数签名，第一种是利用IDA pro 分析so文件，搜索OpenMemory函数：
![IDA获取函数签名](./images/Frida脱壳实战-获取函数签名.png)
### 2.2、方法二
　　在linux系统下使用nm命令直接查看libart.so文件的函数签名：
```shell
$
$ nm libart.so | grep OpenMemory
```
![linux获取函数签名](./images/Frida脱壳实战-Linux获取函数签名.png)

## 三、编写脚本

　　Frida可以执行js脚本也可以执行python脚本：
### 3.1、JS脚本
```js
'use strict';
/**
 * 此脚本在以下环境测试通过
 * android os: 7.1.2 32bit  (64位可能要改OpenMemory的签名)
 * legu: libshella-2.8.so
 * 360:libjiagu.so
 */
Interceptor.attach(Module.findExportByName("libart.so", "_ZN3art7DexFile10OpenMemoryEPKhjRKNSt3__112basic_stringIcNS3_11char_traitsIcEENS3_9allocatorIcEEEEjPNS_6MemMapEPKNS_10OatDexFileEPS9_"), {
    onEnter: function (args) {
      
        //dex起始位置
        var begin = args[1]
        //打印magic
        console.log("magic : " + Memory.readUtf8String(begin))
        //dex fileSize 地址： 
        var address = parseInt(begin,16) + 0x20    //因为dex文件的第三十二个字节处存放的dex文件的大小
        //dex 大小
        var dex_size = Memory.readInt(ptr(address))   //ptr(address)转换的原因是frida只接受NativePointer类型指针

        console.log("dex_size :" + dex_size)
        //dump dex 到/data/data/pkg/目录下
        var file = new File("/data/data/xxx.xxx.xxx/" + dex_size + ".dex", "wb")
        file.write(Memory.readByteArray(begin, dex_size))
        file.flush()
        file.close()
    },
    onLeave: function (retval) {
        if (retval.toInt32() > 0) {
            /* do something */
        }
    }
});
```
### 3.2、Python脚本
```python
#-*- coding:utf-8 -*-
# coding=utf-8
import frida
import sys

def on_message(message, data):
    base = message['payload']['base']
    size = int(message['payload']['size'])
    print(hex(base),size)
    # print session
    # dex_bytes = session.read_bytes(base, size)
    # f = open("1.dex","wb")
    # f.write(dex_bytes)
    # f.close()

# 9.0 arm 需要拦截　_ZN3art13DexFileLoader10OpenCommonEPKhjS2_jRKNSt3__112basic_stringIcNS3_11char_traitsIcEENS3_9allocatorIcEEEEjPKNS_10OatDexFileEbbPS9_NS3_10unique_ptrINS_16DexFileContainerENS3_14default_deleteISH_EEEEPNS0_12VerifyResultE
# 7.0 arm：_ZN3art7DexFile10OpenMemoryEPKhjRKNSt3__112basic_stringIcNS3_11char_traitsIcEENS3_9allocatorIcEEEEjPNS_6MemMapEPKNS_10OatDexFileEPS9_

# android 10: libdexfile.so 
# #_ZN3art13DexFileLoader10OpenCommonEPKhjS2_jRKNSt3__112basic_stringIcNS3_11char_traitsIcEENS3_9allocatorIcEEEEjPKNS_10OatDexFileEbbPS9_NS3_10unique_ptrINS_16DexFileContainerENS3_14default_deleteISH_EEEEPNS0_12VerifyResultE

package = sys.argv[1]
print("dex 导出目录为: /data/data/%s"%(package))
device = frida.get_usb_device()
pid = device.spawn(package)
session = device.attach(pid)
src = """
Interceptor.attach(Module.findExportByName("libdexfile.so", "_ZN3art13DexFileLoader10OpenCommonEPKhjS2_jRKNSt3__112basic_stringIcNS3_11char_traitsIcEENS3_9allocatorIcEEEEjPKNS_10OatDexFileEbbPS9_NS3_10unique_ptrINS_16DexFileContainerENS3_14default_deleteISH_EEEEPNS0_12VerifyResultE"), {
    onEnter: function (args) {
      
        var begin = args[1]
        
        console.log("magic : " + Memory.readUtf8String(begin))
     
        var address = parseInt(begin,16) + 0x20

        var dex_size = Memory.readInt(ptr(address))

        console.log("dex_size :" + dex_size)
      
        var file = new File("/data/data/%s/" + dex_size + ".dex", "wb")
        file.write(Memory.readByteArray(begin, dex_size))
        file.flush()
        file.close()

        var send_data = {}
        send_data.base = parseInt(begin,16)
        send_data.size = dex_size
        send(send_data)
    },
    onLeave: function (retval) {
        if (retval.toInt32() > 0) {
        }
    }
});
"""%(package)

script = session.create_script(src)

script.on("message" , on_message)

script.load()
device.resume(pid)
sys.stdin.read()
```
　　代码参考 https://github.com/dstmath/frida-unpack

## 四、执行脱壳脚本

　　我们先将一个简单的测试程序丢到腾讯乐固上去加固，我们可以看到加固完成后的结果：
![腾讯乐固加固](./images/Frida脱壳实战-腾讯乐固加固.png)
　　可以看到加固过后程序的原本代码被隐藏了，被替换成乐固的壳程序代码，将加固后的应用安装到手机上，并赋予该应用读写sd卡权限。
　　在sd卡中创建unpack目录，执行命令运行JS脚本：
```shell
$
$ frida -U -f  -l  –no-pause
```
　　-U参数表示我们使用了USB server进行链接，-f 参数表示在设备中启动一个指定的android程序，需要配合 –no-pause参数来使进程恢复。-l参数表示需要注入的js脚本。
　　Python脚本的运行就简单得多了：
```shell
$
$ python unpack.py 
```
![Python脱壳](./images/Frida脱壳实战-python脱壳.png)

　　另外注意一个问题，如果应用运行过程中使用了64位的libart.so，脚本的这个地方需要进行一些小修改：
```js
// dex起始位置
var begin = args[1]
// 获取dex起始位置的语句需要改成：
var begin = this.context.x0
```
　　不然可能会出现这样的问题：
![64位libart.so异常](./images/Frida脱壳实战-64位libart.so异常.png)
　　脱壳完成后从sd卡中将unpack文件夹pull到本地：
```shell
$
$ adb pull /sdcard/unpack .
```
![Frida脱壳实战-脱壳后效果](./images/Frida脱壳实战-脱壳后效果.png) 
　　可以看到脱壳成功了。由于这一类脱壳法是将内存中的dex文件直接dump下来的，对于指令抽取等针对方法体进行处理的加固方式并没有做对应的修复工作。如果应用经过了指令抽取或者dex2c处理，这种方法拿到的dex文件就是残缺不全的，对于那些加固就需要通过对Android源码进行修改的脱壳方法比如dexhunter或者是比较新的FART脱壳机。
